### Modeling

#### Imports

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
import xgboost as xgb

#### Data Preprocessing 

- TO DO: HANDLING NULL-VALUES 
- ADDING INTERACTION TERMS (poly features)

In [19]:
def load_data():
    # Load train and set
    X_train     = pd.read_csv('X_train.csv', index_col='ID')
    Y_train     = pd.read_csv('Y_train.csv', index_col='ID')
    X_test      = pd.read_csv('X_test.csv', index_col='ID')
    Y_test      = pd.read_csv('Y_test.csv', index_col='ID')

    # Join features and target for preprocessing
    train_df    = X_train.join(Y_train)
    test_df     = X_test.join(Y_test)

    # Split training data into DE and FR datasets ||| Forward fill missing values
    train_fr    = train_df[train_df.COUNTRY=='FR'].sort_values(by='DAY_ID').ffill().fillna(0)
    train_de    = train_df[train_df.COUNTRY=='DE'].sort_values(by='DAY_ID').ffill().fillna(0)

    # Split test data into DE and FR datasets ||| Forward fill missing values
    test_fr     = test_df[test_df.COUNTRY=='FR'].sort_values(by='DAY_ID').ffill().fillna(0)
    test_de     = test_df[test_df.COUNTRY=='DE'].sort_values(by='DAY_ID').ffill().fillna(0)

    return [train_fr, train_de, test_fr, test_de]

In [20]:
train_fr, train_de, test_fr, test_de = load_data()

In [21]:
def add_target_lagged(df):
    for i in range(0,13,1):
        df[f'TARGET_LAG{i}'] = df['TARGET'].shift(-(i+1)).ffill()
    return df

In [22]:
def add_lagged_features(df):
    cols = ['DE_CONSUMPTION', 'FR_CONSUMPTION',
    'DE_FR_EXCHANGE', 'FR_DE_EXCHANGE', 'DE_NET_EXPORT', 'FR_NET_EXPORT',
    'DE_NET_IMPORT', 'FR_NET_IMPORT', 'DE_GAS', 'FR_GAS', 'DE_COAL',
    'FR_COAL', 'DE_HYDRO', 'FR_HYDRO', 'DE_NUCLEAR', 'FR_NUCLEAR',
    'DE_SOLAR', 'FR_SOLAR', 'DE_WINDPOW', 'FR_WINDPOW', 'DE_LIGNITE',
    'DE_RESIDUAL_LOAD', 'FR_RESIDUAL_LOAD', 'DE_RAIN', 'FR_RAIN', 'DE_WIND',
    'FR_WIND', 'DE_TEMP', 'FR_TEMP', 'GAS_RET', 'COAL_RET', 'CARBON_RET']

    for col in cols: 
        for i in range(0,13,1):
            df[f'{col}_LAG{i}'] = df[col].shift(-(i+1)).ffill()
    return df

In [23]:
train_fr = add_target_lagged(train_fr)
test_fr = add_target_lagged(test_fr)
train_fr = add_lagged_features(train_fr)
test_fr = add_lagged_features(test_fr)

/var/folders/ch/x5l07kfd371389w3bp4rxwpc0000gn/T/ipykernel_28005/2289295321.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_LAG{i}'] = df[col].shift(-(i+1)).ffill()
/var/folders/ch/x5l07kfd371389w3bp4rxwpc0000gn/T/ipykernel_28005/2289295321.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_LAG{i}'] = df[col].shift(-(i+1)).ffill()
/var/folders/ch/x5l07kfd371389w3bp4rxwpc0000gn/T/ipykernel_28005/2289295321.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of callin

In [24]:
Y_train_fr  = train_fr.TARGET
X_train_fr  = train_fr.drop(columns=['TARGET','DAY_ID','COUNTRY'])
Y_test_fr   = test_fr.TARGET
X_test_fr   = test_fr.drop(columns=['TARGET','DAY_ID','COUNTRY'])

In [25]:
train_fr.shape

(851, 464)

In [26]:
test_fr.shape

(365, 464)

In [27]:
rf = RandomForestRegressor()
rf.fit(X_train_fr,Y_train_fr)

RandomForestRegressor()

In [28]:
from sklearn.metrics import mean_squared_error

In [29]:
mean_squared_error(Y_test_fr, rf.predict(X_test_fr))

1.234642040974661

In [17]:
#pd.DataFrame(index=rf.feature_names_in_, data=rf.feature_importances_).sort_values(by=0,ascending=False).plot(kind='bar')
pd.DataFrame(index=rf.feature_names_in_, data=rf.feature_importances_).sort_values(by=0,ascending=False)

,0
FR_SOLAR_LAG2,0.060660
DE_TEMP_LAG0,0.030368
FR_RESIDUAL_LOAD_LAG8,0.021692
GAS_RET_LAG11,0.020461
DE_NUCLEAR_LAG3,0.014675
...,...
FR_DE_EXCHANGE_LAG10,0.000177
FR_RAIN_LAG7,0.000163
FR_NET_IMPORT_LAG6,0.000147
DE_FR_EXCHANGE_LAG0,0.000130


#### Hyperparameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Define parameter grids for hyperparameter tuning
param_grid_decision_tree = {'max_depth': [None, 10, 20, 30],
                            'min_samples_split': [2, 5, 10]}

param_grid_adaboost = {'n_estimators': [50, 100, 200],
                       'learning_rate': [0.01, 0.1, 1.0]}

param_grid_bagging = {'n_estimators': [10, 50, 100],
                      'max_samples': [0.5, 0.8, 1.0],
                      'max_features': [0.5, 0.8, 1.0]}

param_grid_gradient_boosting = {'n_estimators': [50, 100, 200],
                                'learning_rate': [0.01, 0.1, 1.0],
                                'max_depth': [3, 5, 10]}

param_grid_random_forest = {'n_estimators': [50, 100, 200],
                            'max_depth': [None, 10, 20],
                            'min_samples_split': [2, 5, 10]}

param_grid_extra_trees = {'n_estimators': [50, 100, 200],
                          'max_depth': [None, 10, 20],
                          'min_samples_split': [2, 5, 10]}

# Initialize models
decision_tree = DecisionTreeRegressor()
adaboost = AdaBoostRegressor(base_estimator=decision_tree)
bagging = BaggingRegressor(base_estimator=decision_tree)
gradient_boosting = GradientBoostingRegressor()
random_forest = RandomForestRegressor()
extra_trees = ExtraTreesRegressor()

# List of models and corresponding parameter grids
models = [(decision_tree, param_grid_decision_tree),
          (adaboost, param_grid_adaboost),
          (bagging, param_grid_bagging),
          (gradient_boosting, param_grid_gradient_boosting),
          (random_forest, param_grid_random_forest),
          (extra_trees, param_grid_extra_trees)]

# Fit and tune each model
for model, param_grid in models:
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, Y_train)
    
    # Best hyperparameters
    best_params = grid_search.best_params_
    
    # Best model
    best_model = grid_search.best_estimator_
    
    # Evaluate on test set
    predictions = best_model.predict(X_test)
    mse = mean_squared_error(Y_test, predictions)
    
    # Print results
    print("Model:", model.__class__.__name__)
    print("Best Parameters:", best_params)
    print("Mean Squared Error (Test):", mse)
    print("\n")


Model: DecisionTreeRegressor
Best Parameters: {'max_depth': 10, 'min_samples_split': 10}
Mean Squared Error (Test): 1.4903456298393765




/Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  w

Model: AdaBoostRegressor
Best Parameters: {'learning_rate': 1.0, 'n_estimators': 100}
Mean Squared Error (Test): 1.6936049692508026




/Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_bagging.py:509: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_bagging.py:509: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.

Model: BaggingRegressor
Best Parameters: {'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 100}
Mean Squared Error (Test): 1.1633569364971386




/Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was pass

KeyboardInterrupt: 